# PRB-Graph: Resource Management Reconstruction GNN  
## CPSC 8810 — ML4G Term Project
### Authored by Ryan Barker

This notebook implements a minimal viable **PRB-Graph** model: an offline, UE-level graph neural network that reconstructs resource-management behavior from noisy per-TTI proportional-fair (PF) scheduler logs. Using the existing TTI-Trust Anamoly Detector + Classifier preprocessing pipeline, each sliding window of TTIs is converted into a **UE-contention graph** whose nodes represent UEs active in the window and whose edges capture co-scheduling (PRB contention) relationships. Node features summarize PRB usage statistics, fairness/PF surrogates, and starvation patterns, while a compact **GraphSAGE → global pooling → MLP** architecture performs **graph-level classification** to distinguish benign scheduling behavior from PF-based PRB starvation indicative of attack conditions. This MVP focuses on methodology rather than headline accuracy, acknowledging the dataset’s timing noise and simulator artifacts, and serves as a prototype for future training on GPU-native, slot-accurate Aerial/cuMAC traces.

> <span style="color:red; font-weight:bold">Disclaimer — Prototype PRB-Graph Results</span>  
>  
> The PRB-Graph results in this notebook were obtained **only on a small, noisy prototype dataset** derived from the original ~30 GB Open Air Interface simulator logs. Due to known timing noise within the low-latency experiment which generated the data, label corruption, and severe class imbalance in this dataset, **the model was deliberately *not* tuned or optimized to the full data volume**.  
>  
> These experiments are intended to validate the **model architecture and end-to-end graph construction pipeline only**. The reported metrics (including the best configuration  
> `hidden_dim=32, num_layers=1, dropout=0.0, conv_type="sage", aggr="mean", lr=2e-4` with `F1 = 0.0`) **should not be interpreted as meaningful performance claims for PRB starvation detection in real RAN systems**.  
>  
> A proper hyperparameter search and performance evaluation will be conducted **only after more accurate PF ledgers are collected from the new NVIDIA AI RAN stack**, at which point the same PRB-GraphSAGE code will be re-used on cleaner, slot-accurate traces. The limitations of the current dataset and their impact on model behavior will be discussed in detail in the final report.

### preproc_tti_trust.py

This script is a preprocessing scaffold for the TTI-Trust project that converts lightweight attack/benign CSV “shims” into parquet datasets, derives scheduler-native, identity-agnostic features, and emits windowed shards plus cross-validation splits for PyTorch. It first materializes CSVs into partitioned parquet (by `run_id`), adding a relative time axis `sec_rel` if needed, then for each run it auto-detects UE resource-block columns, computes `total_rb`, normalizes RB usage into per-UE PRB shares, and derives a Jain’s fairness index `J`. On top of these per-UE shares it builds “roles” by tracking an exponential-moving-average of the shares, selecting the top-K dominant UEs per TTI, and aggregating everyone else into a `rest_share` and corresponding “small RB” indicators, plus 1-second presence flags to capture coarse duty cycle behavior. Using these role-based features and the `phase` labels, it slides long and short windows (e.g., 240- and 64-TTI) with multiple strides, labels each window as benign, attack, or ambiguous via a dominance test on the top role (with thresholds like `ATTACK_DOMINANCE`, `OTHERS_MAX`, and `DUTY_KEEP`), and batches them into compressed NumPy `.npz` shards alongside parquet meta tables that record window boundaries, labels, and summary statistics. The script writes enriched per-run parquet files, then populates a `win_shards` directory with both long and short window shards for attack and benign sources. Finally, it builds grouped K-fold splits where entire `run_id`s are kept together in either train or validation, returning index pairs for each fold and printing a brief summary so downstream training can stream shards or assemble cross-validation sets directly from the generated metadata.

In [1]:
# TTI-Trust preprocessing scaffold for PyTorch: identity-agnostic feature composer,
# phase-aware windowing, and grouped K-fold splits.

from __future__ import annotations
import os, re, warnings, itertools
from dataclasses import dataclass
from typing import List, Tuple
from glob import glob

import numpy as np
import pandas as pd
import pyarrow as pa, pyarrow.csv as pv, pyarrow.parquet as pq

# -------------------- DATASETS --------------------
ATTACK_FN = "data/full_dataset/prb_tti_evidence_ai_attack.csv" # Committing lightweight shims to the repo due to storage quotas, full 15GB dataset available upon request
BENIGN_FN = "data/full_dataset/prb_tti_evidence_ai_benign.csv"  # Committing lightweight shims to the repo due to storage quotas, full 15GB dataset available upon request

# -------------------- Constants aligned to paper --------------------
TTI_SEC = 0.0005
C_PRB   = 106

W_LONG  = 240
W_SHORT = 64

STRIDES_LONG  = [8, 12]
STRIDES_SHORT = [4, 6]

TOP_K = 6
SMALL_RB_EPS = 4

ATTACK_DOMINANCE = 0.90
OTHERS_MAX       = 0.10
DUTY_KEEP        = 0.90

# -------------------- Helpers --------------------

def csv_to_parquet(src_csv: str, dest_root: str):
    os.makedirs(dest_root, exist_ok=True)
    for _root, _dirs, files in os.walk(dest_root):
        if any(f.endswith(".parquet") for f in files):
            return

    convert_cols = {"run_id": pa.large_string(), "phase": pa.large_string()}
    read_opts  = pv.ReadOptions(block_size=1<<26)
    conv_opts  = pv.ConvertOptions(column_types=convert_cols, strings_can_be_null=True)
    t = pv.read_csv(src_csv, read_options=read_opts, convert_options=conv_opts)

    if "run_id" not in t.schema.names:
        stem = os.path.splitext(os.path.basename(src_csv))[0]
        run_id = pa.array(np.full(t.num_rows, stem), type=pa.large_string())
        t = t.append_column("run_id", run_id)

    if "sec_rel" not in t.schema.names:
        n = t.num_rows
        sec_rel = pa.array((np.arange(n, dtype=np.int32) * np.float32(TTI_SEC)), type=pa.float32())
        t = t.append_column("sec_rel", sec_rel)

    pq.write_to_dataset(t, root_path=dest_root, partition_cols=["run_id"],
                        compression="zstd", use_dictionary=True)

def _detect_ue_rb_columns(df: pd.DataFrame) -> List[str]:
    cand = [c for c in df.columns if re.fullmatch(r'UE\d+_rb', c, flags=re.IGNORECASE)]
    if not cand:
        cand = [c for c in df.columns if c.lower().endswith('_rb')]
    def _key(c):
        m = re.search(r'(\d+)', c)
        return int(m.group(1)) if m else 10**9
    return sorted(set(cand), key=_key)

def _ensure_total_rb(df: pd.DataFrame, ue_cols: List[str]) -> pd.DataFrame:
    if 'total_rb' not in df.columns:
        df['total_rb'] = df[ue_cols].sum(axis=1)
    return df

def _compute_shares_and_fairness(df: pd.DataFrame, ue_cols: List[str]) -> pd.DataFrame:
    shares = (df[ue_cols].clip(lower=0).astype('float32') / np.float32(C_PRB))
    shares.columns = [c.replace('_rb','_share') for c in ue_cols]
    for c in shares.columns:
        df[c] = shares[c]
    x = shares.to_numpy(dtype='float32', copy=False)
    sum_x  = x.sum(axis=1, dtype='float32')
    sum_x2 = (x*x).sum(axis=1, dtype='float32')
    n = np.float32(max(1, len(ue_cols)))
    with np.errstate(divide='ignore', invalid='ignore'):
        J = (sum_x*sum_x) / (n * sum_x2)
    df['J'] = np.nan_to_num(J, nan=0.0, posinf=0.0, neginf=0.0).astype('float32')
    return df

def _compose_roles(df: pd.DataFrame, ue_cols: List[str], top_k: int = TOP_K) -> pd.DataFrame:
    share_cols = [c.replace('_rb','_share') for c in ue_cols]
    alpha = 2.0 / (20 + 1.0)
    for c in share_cols:
        df[c+"_ema"] = df[c].astype('float32').ewm(alpha=alpha, adjust=False).mean().astype('float32')

    S   = df[share_cols].to_numpy(dtype='float32', copy=False)
    E   = df[[c+"_ema" for c in share_cols]].to_numpy(dtype='float32', copy=False)
    PRB = df[ue_cols].to_numpy(dtype='int32',    copy=False)

    T, U = S.shape
    k = min(top_k, U)
    order  = np.argsort(-E, axis=1)
    top_ix = order[:, :k]

    roles       = np.take_along_axis(S,   top_ix, axis=1).astype('float32')
    roles_small = np.take_along_axis((PRB < SMALL_RB_EPS).astype('float32'), top_ix, axis=1)

    sum_all    = S.sum(axis=1, dtype='float32')
    sum_topk   = roles.sum(axis=1, dtype='float32')
    rest_share = (sum_all - sum_topk).clip(min=0).astype('float32')

    if U > k:
        row_ix = np.arange(T)[:, None]
        mask = np.ones_like(S, dtype=bool); mask[row_ix, top_ix] = False
        cnt  = mask.sum(axis=1)
        rest_small = ((PRB < SMALL_RB_EPS).astype('float32') * mask).sum(axis=1) / np.maximum(cnt, 1)
    else:
        rest_small = np.zeros(T, dtype='float32')

    for i in range(k):
        df[f'role{i+1}_share']   = roles[:, i]
        df[f'role{i+1}_smallrb'] = roles_small[:, i]
    for i in range(k, top_k):
        df[f'role{i+1}_share']   = np.float32(0.0)
        df[f'role{i+1}_smallrb'] = np.float32(0.0)
    df['rest_share']   = rest_share
    df['rest_smallrb'] = rest_small

    sec_bin = np.floor(df['sec_rel'].to_numpy(dtype='float32')).astype('int32')
    max_sec = int(sec_bin.max(initial=0))
    def sec_presence(vec_bool: np.ndarray) -> np.ndarray:
        agg = np.zeros(max_sec + 1, dtype='uint8')
        np.maximum.at(agg, sec_bin, vec_bool.astype('uint8'))
        return agg[sec_bin]
    for i in range(top_k):
        df[f'role{i+1}_present_1s'] = sec_presence((df[f'role{i+1}_share'].to_numpy() > 0))
    df['rest_present_1s'] = sec_presence((df['rest_share'].to_numpy() > 0))
    return df

def _label_window(shares_mat: np.ndarray) -> bool:
    if shares_mat.size == 0:
        return False
    top    = shares_mat[:, 0]
    others = shares_mat[:, 1:]
    ok = (top >= ATTACK_DOMINANCE) & (np.max(others, axis=1) <= OTHERS_MAX)
    return (ok.mean() >= DUTY_KEEP)

@dataclass
class WindowSpec:
    length_tti: int
    stride_tti: int

def iter_windows(df: pd.DataFrame, run_id: str, spec: WindowSpec, role_cols: List[str],
                 phases_attack={"attack"}, phases_benign={"baseline","recovery","benign"},
                 batch_windows: int = 8192):
    W, S = spec.length_tti, spec.stride_tti
    R = df[role_cols].to_numpy(dtype='float32', copy=False)
    phase = df['phase'].astype(str).to_numpy()
    T = len(df)
    starts = np.arange(0, max(0, T - W + 1), S, dtype=np.int64)

    bufX, rows = [], []
    for s in starts:
        e = s + W
        blk = R[s:e, :]
        if blk.shape[0] != W:
            continue
        win_phase = phase[s:e]
        frac_attack = np.isin(win_phase, list(phases_attack)).mean()
        frac_benign = np.isin(win_phase, list(phases_benign)).mean()
        looks_attacky = _label_window(blk)
        if (frac_attack >= 0.90) and looks_attacky: y = 2
        elif (frac_benign >= 0.90):                y = 0
        else:                                      y = 1

        bufX.append(blk)
        rows.append((run_id, int(s), int(e), int(W), int(y),
                     float(frac_attack), float(frac_benign), int(looks_attacky)))

        if len(bufX) == batch_windows:
            X = np.stack(bufX, axis=0)
            meta = pd.DataFrame.from_records(
                rows,
                columns=["run_id","start_tti","end_tti","window_len","label",
                         "frac_attack","frac_benign","looks_attacky"]
            )
            yield X, meta
            bufX.clear(); rows.clear()

    if bufX:
        X = np.stack(bufX, axis=0)
        meta = pd.DataFrame.from_records(
            rows,
            columns=["run_id","start_tti","end_tti","window_len","label",
                     "frac_attack","frac_benign","looks_attacky"]
        )
        yield X, meta

# -------------------- Main entry: build datasets (partitioned, low RAM) --------------------

csv_to_parquet(ATTACK_FN, "parquet/attack")
csv_to_parquet(BENIGN_FN, "parquet/benign")

def iter_run_partitions(root: str):
    for run_dir in sorted(glob(os.path.join(root, "run_id=*"))):
        rid = os.path.basename(run_dir).split("=", 1)[1]
        df = pd.read_parquet(run_dir, engine="pyarrow")
        if 'run_id' not in df.columns:
            df['run_id'] = rid
        if 'phase' not in df.columns:
            df['phase'] = 'unknown'
        if 'sec_rel' not in df.columns:
            df['sec_rel'] = (np.arange(len(df), dtype=np.int32) * np.float32(TTI_SEC)).astype('float32')
        yield rid, df

def process_source(root: str, src_name: str):
    if not os.path.isdir(root):
        print(f"[{src_name}] No parquet root found at {root} (skipping).")
        return

    os.makedirs("win_shards", exist_ok=True)
    shard_counter = itertools.count(0)

    runs = list(iter_run_partitions(root))
    if not runs:
        print(f"[{src_name}] No run_id partitions found in {root}.")
        return
    print(f"[{src_name}] Found {len(runs)} run_id partitions.")

    for rid, df in runs:
        ue_cols = _detect_ue_rb_columns(df)
        if not ue_cols:
            warnings.warn(f"[{src_name}] run_id={rid}: no UE *_rb columns; skipping")
            continue

        df = _ensure_total_rb(df, ue_cols)
        df = _compute_shares_and_fairness(df, ue_cols)
        df = _compose_roles(df, ue_cols, top_k=TOP_K)

        keep = [f'role{i}_share' for i in range(1, TOP_K+1)] + ['rest_share','J','sec_rel','run_id','phase']
        enriched_out = f"{src_name}_enriched_{rid}.parquet"
        df[keep].to_parquet(enriched_out, index=False)
        print(f"[{src_name}] run_id={rid} → {enriched_out}  (rows={len(df):,})")

        role_cols = [f'role{k}_share' for k in range(1, TOP_K+1)] + ['rest_share']
        g = df.reset_index(drop=True)

        for stride in STRIDES_LONG:
            spec = WindowSpec(length_tti=W_LONG, stride_tti=stride)
            for X, meta in iter_windows(g, rid, spec, role_cols):
                sid = next(shard_counter)
                np.savez_compressed(f"win_shards/long_s{stride}_{src_name}_{rid}_{sid}.npz", X=X)
                meta["source"] = src_name; meta["stride"] = stride; meta["kind"] = "long"
                meta.to_parquet(f"win_shards/long_s{stride}_{src_name}_{rid}_{sid}_meta.parquet", index=False)

        for stride in STRIDES_SHORT:
            spec = WindowSpec(length_tti=W_SHORT, stride_tti=stride)
            for X, meta in iter_windows(g, rid, spec, role_cols):
                sid = next(shard_counter)
                np.savez_compressed(f"win_shards/short_s{stride}_{src_name}_{rid}_{sid}.npz", X=X)
                meta["source"] = src_name; meta["stride"] = stride; meta["kind"] = "short"
                meta.to_parquet(f"win_shards/short_s{stride}_{src_name}_{rid}_{sid}_meta.parquet", index=False)

process_source("parquet/attack", "attack")
process_source("parquet/benign", "benign")

def load_all_meta(kind: str) -> pd.DataFrame:
    metas = [pd.read_parquet(p) for p in glob(f"win_shards/{kind}_*_meta.parquet")]
    return pd.concat(metas, ignore_index=True) if metas else pd.DataFrame()

M_long  = load_all_meta("long")
M_short = load_all_meta("short")

def grouped_kfold(meta: pd.DataFrame, K: int = 5, seed: int = 42) -> List[Tuple[np.ndarray, np.ndarray]]:
    if meta.empty:
        return []
    rng = np.random.default_rng(seed)
    run_ids = meta['run_id'].astype(str).unique().tolist()
    rng.shuffle(run_ids)
    folds = [set() for _ in range(K)]
    for i, rid in enumerate(run_ids):
        folds[i % K].add(rid)
    splits = []
    for i in range(K):
        val_rids = folds[i]
        train_idx = meta.index[~meta['run_id'].astype(str).isin(val_rids)].to_numpy()
        val_idx   = meta.index[ meta['run_id'].astype(str).isin(val_rids)].to_numpy()
        splits.append((train_idx, val_idx))
    return splits

splits_long  = grouped_kfold(M_long,  K=5, seed=2025)
splits_short = grouped_kfold(M_short, K=5, seed=2025)

if len(M_long):
    print(f"\n[long] meta={M_long.shape}, groups={M_long['run_id'].nunique()}, folds={len(splits_long)}")
if len(M_short):
    print(f"[short] meta={M_short.shape}, groups={M_short['run_id'].nunique()}, folds={len(splits_short)}")

print("\nPreprocessing complete. Shards are in ./win_shards; use meta to assemble CV splits or stream for training.")

[attack] Found 13 run_id partitions.
[attack] run_id=040933 → attack_enriched_040933.parquet  (rows=1,388,017)
[attack] run_id=171614 → attack_enriched_171614.parquet  (rows=762,483)
[attack] run_id=173450 → attack_enriched_173450.parquet  (rows=1,470,117)
[attack] run_id=175206 → attack_enriched_175206.parquet  (rows=2,219,957)
[attack] run_id=181054 → attack_enriched_181054.parquet  (rows=2,947,494)
[attack] run_id=183046 → attack_enriched_183046.parquet  (rows=2,045,825)
[attack] run_id=185206 → attack_enriched_185206.parquet  (rows=4,292,603)
[attack] run_id=190930 → attack_enriched_190930.parquet  (rows=4,962,590)
[attack] run_id=192814 → attack_enriched_192814.parquet  (rows=5,550,536)
[attack] run_id=194737 → attack_enriched_194737.parquet  (rows=1,450,956)
[attack] run_id=200821 → attack_enriched_200821.parquet  (rows=6,819,582)
[attack] run_id=202815 → attack_enriched_202815.parquet  (rows=7,485,965)
[attack] run_id=204929 → attack_enriched_204929.parquet  (rows=1,227,412)
[be

### Data Loader

This code builds a full PyTorch Geometric pipeline that turns PRB time-series windows into UE-level contention graphs and then into train/validation loaders for a PRB-GraphSAGE model. It starts by detecting UE RB columns (e.g., `UE1_rb`) in per-run parquet “PF ledger” data, then constructs node features for each UE in a window using PRB-based statistics such as mean/min/max/std share of PRBs, activity and zero fractions, and the longest zero (starvation) run. Edges are created between UEs that are co-scheduled at least once in the window, yielding an undirected contention graph. For each window in the preprocessed metadata (`win_shards/{kind}_*_meta.parquet`), `build_graph_from_window` slices the run dataframe, computes node features and edges, and assigns a graph label (binary or multiclass) from the window label, packaging everything into a `torch_geometric.data.Data` object plus a compact metadata dict. `build_prb_graphs` loads and filters all meta files, caches run-level parquet data and UE column detection by `(source, run_id)`, iterates over windows to construct graphs, and returns the list of graphs along with a metadata DataFrame. On top of this, `make_prb_graph_loaders` performs a grouped train/validation split by `run_id` (so windows from the same run don’t leak across splits), builds `GeoDataLoader` instances for training and validation, and returns them alongside split indices and run ID sets. Finally, the script instantiates these loaders for short windows, infers the node feature dimensionality with `detect_node_dim`, and prints it, providing everything needed to feed PRB-GraphSAGE with graph-structured inputs derived from scheduler PRB logs.

In [2]:
import os, re, glob
from collections import OrderedDict
from typing import Optional, List, Tuple

import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, Subset
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader as GeoDataLoader

import pyarrow.dataset as ds 

C_PRB = 106


def _detect_ue_rb_columns_from_names(names: List[str]) -> List[str]:
    cand = [c for c in names if re.fullmatch(r'UE\d+_rb', c, flags=re.IGNORECASE)]
    if not cand:
        cand = [c for c in names if c.lower().endswith("_rb")]

    def _key(c):
        m = re.search(r"(\d+)", c)
        return int(m.group(1)) if m else 10**9

    return sorted(set(cand), key=_key)


def _longest_zero_run(arr: np.ndarray) -> float:
    longest = 0
    current = 0
    for v in arr:
        if v == 0:
            current += 1
            longest = max(longest, current)
        else:
            current = 0
    return float(longest)


def build_node_features(prb_window: np.ndarray) -> np.ndarray:
    # avoid extra copies when already float32
    prb = np.asarray(prb_window, dtype=np.float32)  # view if possible
    W, U = prb.shape
    if U == 0:
        return np.zeros((0, 1), dtype=np.float32)

    share = prb / np.float32(C_PRB)

    mean_share = share.mean(axis=0)
    min_share = share.min(axis=0)
    max_share = share.max(axis=0)
    std_share = share.std(axis=0)

    active = (prb > 0).astype(np.float32)
    active_frac = active.mean(axis=0)
    zero_frac = (prb == 0).mean(axis=0)
    longest_zero = np.array([_longest_zero_run(prb[:, u]) for u in range(U)], dtype=np.float32)

    return np.stack(
        [mean_share, min_share, max_share, std_share, active_frac, zero_frac, longest_zero],
        axis=1
    ).astype(np.float32)


def build_edges_fast(prb_window: np.ndarray) -> torch.Tensor:
    """
    Vectorized replacement for the O(U^2*W) Python double-loop.
    Builds undirected edges where two UEs are co-scheduled at least once.
    """
    prb = np.asarray(prb_window)
    W, U = prb.shape
    if U <= 1:
        return torch.empty((2, 0), dtype=torch.long)

    active = (prb > 0)
    # counts of co-activity: cast so matmul counts instead of bool logic
    a = active.astype(np.int16, copy=False)
    co = a.T @ a  # [U, U] int16, co[i,j] = #TTIs both active
    # keep upper triangle (no self loops), then mirror to make directed pairs
    u, v = np.nonzero(np.triu(co, 1))
    if u.size == 0:
        return torch.empty((2, 0), dtype=torch.long)

    src = np.concatenate([u, v]).astype(np.int64, copy=False)
    dst = np.concatenate([v, u]).astype(np.int64, copy=False)
    return torch.from_numpy(np.stack([src, dst], axis=0))


class PRBWindowDataset(Dataset):
    """
    Lazy, disk-backed dataset:
      - stores only window metadata in RAM
      - loads ONE run parquet at a time (LRU cache) and slices windows from it
      - builds graph on demand in __getitem__
    """
    def __init__(
        self,
        kind: str = "short",
        meta_glob: Optional[str] = None,
        parquet_root_attack: str = "parquet/attack",
        parquet_root_benign: str = "parquet/benign",
        label_mode: str = "binary",
        restrict_sources: Optional[List[str]] = None,
        cache_max_runs: int = 1,   # keep this small; each worker has its own cache
    ):
        super().__init__()
        self.kind = kind
        self.parquet_root_attack = parquet_root_attack
        self.parquet_root_benign = parquet_root_benign
        self.label_mode = label_mode
        self.restrict_sources = restrict_sources
        self.cache_max_runs = int(cache_max_runs)

        if meta_glob is None:
            meta_glob = f"win_shards/{kind}_*_meta.parquet"
        self.meta_files = sorted(glob.glob(meta_glob))
        if not self.meta_files:
            raise FileNotFoundError(f"No meta parquet files found for pattern: {meta_glob}")

        # Load ONLY minimal meta columns
        needed = ["run_id", "source", "start_tti", "end_tti", "label", "kind", "stride"]
        parts = []
        for mp in self.meta_files:
            m = pd.read_parquet(mp)
            # keep only columns that exist
            keep = [c for c in needed if c in m.columns]
            m = m[keep].copy()
            if restrict_sources is not None and "source" in m.columns:
                m = m[m["source"].isin(restrict_sources)]
            if len(m):
                parts.append(m)

        if not parts:
            raise RuntimeError("No meta rows found after filtering.")

        meta = pd.concat(parts, ignore_index=True)

        # basic sanitation
        meta["start_tti"] = meta["start_tti"].astype(np.int32)
        meta["end_tti"] = meta["end_tti"].astype(np.int32)
        if "label" not in meta.columns:
            meta["label"] = 0
        meta["label"] = meta["label"].astype(np.int8)

        # store run_id/source as categoricals to avoid repeating strings N times
        run_cat = pd.Categorical(meta["run_id"].astype(str))
        src_cat = pd.Categorical(meta["source"].astype(str))

        self._run_categories = run_cat.categories.tolist()
        self._src_categories = src_cat.categories.tolist()
        self.run_code = run_cat.codes.astype(np.int32)
        self.src_code = src_cat.codes.astype(np.int8)
        self.start = meta["start_tti"].to_numpy(np.int32)
        self.end = meta["end_tti"].to_numpy(np.int32)
        self.label = meta["label"].to_numpy(np.int8)

        # tiny LRU cache: (src_str, run_id_str) -> (prb_run_uint8 [T,U], U)
        self._run_cache: "OrderedDict[Tuple[str,str], np.ndarray]" = OrderedDict()

    def __len__(self) -> int:
        return int(self.start.shape[0])

    def _run_path(self, source: str, run_id: str) -> str:
        root = self.parquet_root_benign
        if source == "attack":
            root = self.parquet_root_attack
        elif source == "benign":
            root = self.parquet_root_benign

        run_dir = os.path.join(root, f"run_id={run_id}")
        if os.path.exists(run_dir):
            return run_dir
        if os.path.exists(run_dir + ".parquet"):
            return run_dir + ".parquet"
        raise FileNotFoundError(f"Missing parquet for source={source}, run_id={run_id}: {run_dir}")

    def _load_run_prb_matrix(self, source: str, run_id: str) -> np.ndarray:
        key = (source, run_id)
        if key in self._run_cache:
            self._run_cache.move_to_end(key)
            return self._run_cache[key]

        path = self._run_path(source, run_id)
        dataset = ds.dataset(path, format="parquet")
        ue_cols = _detect_ue_rb_columns_from_names(dataset.schema.names)
        if not ue_cols:
            raise RuntimeError(f"No UE*_rb columns found in run parquet: {path}")

        table = dataset.to_table(columns=ue_cols).combine_chunks()

        T = table.num_rows
        U = len(ue_cols)
        prb = np.empty((T, U), dtype=np.uint8)
        for k, c in enumerate(ue_cols):
            col = table[c].to_numpy(zero_copy_only=False)
            if np.issubdtype(col.dtype, np.floating):
                col = np.nan_to_num(col, nan=0.0)
            prb[:, k] = col.astype(np.uint8, copy=False)

        self._run_cache[key] = prb
        self._run_cache.move_to_end(key)
        while len(self._run_cache) > self.cache_max_runs:
            self._run_cache.popitem(last=False)  # evict LRU

        return prb

    def __getitem__(self, idx: int) -> Data:
        run_id = self._run_categories[int(self.run_code[idx])]
        source = self._src_categories[int(self.src_code[idx])]

        prb_run = self._load_run_prb_matrix(source, run_id)
        s = int(self.start[idx])
        e = int(self.end[idx])
        if e <= s:
            # return safe empty graph (won't crash collate)
            y = torch.tensor(0, dtype=torch.long)
            return Data(
                x=torch.zeros((0, 7), dtype=torch.float32),
                edge_index=torch.empty((2, 0), dtype=torch.long),
                y=y,
            )

        prb_window = prb_run[s:e, :]  # view into uint8 run matrix

        x = torch.from_numpy(build_node_features(prb_window))
        edge_index = build_edges_fast(prb_window)

        label_raw = int(self.label[idx])
        if self.label_mode == "binary":
            y = torch.tensor(1 if label_raw == 2 else 0, dtype=torch.long)
        elif self.label_mode == "multiclass":
            y = torch.tensor(label_raw, dtype=torch.long)
        else:
            raise ValueError(f"Unknown label_mode: {self.label_mode}")

        return Data(x=x, edge_index=edge_index, y=y)

def make_prb_graph_loaders_lazy(
    kind: str = "short",
    batch_size: int = 32,
    num_workers: int = 0,
    seed: int = 2025,
    val_ratio: float = 0.2,
    label_mode: str = "binary",
    restrict_sources: Optional[List[str]] = None,
    cache_max_runs: int = 1,
):
    ds_all = PRBWindowDataset(
        kind=kind,
        label_mode=label_mode,
        restrict_sources=restrict_sources,
        cache_max_runs=cache_max_runs,
    )

    # group split by run_id code
    run_codes = np.unique(ds_all.run_code)
    rng = np.random.default_rng(seed)
    rng.shuffle(run_codes)
    n_val = max(1, int(len(run_codes) * val_ratio))
    val_codes = set(run_codes[:n_val].tolist())

    idx_all = np.arange(len(ds_all), dtype=np.int64)
    is_val = np.isin(ds_all.run_code, np.array(list(val_codes), dtype=np.int32))
    val_idx = idx_all[is_val].tolist()
    train_idx = idx_all[~is_val].tolist()

    train_ds = Subset(ds_all, train_idx)
    val_ds = Subset(ds_all, val_idx)

    train_loader = GeoDataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    val_loader = GeoDataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    return train_loader, val_loader

def make_baseline_loaders_from_ds(
    ds_all,
    batch_size=256,
    seed=2025,
    keep_attack_runs=2,
    keep_benign_runs=2,
    max_windows_total=300_000,
    val_ratio=0.2,
    num_workers=0,
):
    rng = np.random.default_rng(seed)
    src_to_code = {s:i for i,s in enumerate(ds_all._src_categories)}
    run_to_code = {r:i for i,r in enumerate(ds_all._run_categories)}

    def first_n_runs(source, n):
        sc = src_to_code[source]
        rcs = np.unique(ds_all.run_code[ds_all.src_code == sc])
        rids = sorted(ds_all._run_categories[int(rc)] for rc in rcs)
        return rids[:n]

    keep_run_ids = first_n_runs("attack", keep_attack_runs) + first_n_runs("benign", keep_benign_runs)
    keep_codes = np.array([run_to_code[r] for r in keep_run_ids], dtype=np.int32)

    idx = np.flatnonzero(np.isin(ds_all.run_code, keep_codes))

    # downsample windows
    if max_windows_total is not None and idx.size > max_windows_total:
        idx = rng.choice(idx, size=max_windows_total, replace=False)

    # grouped split by run_id (still)
    idx_run_codes = ds_all.run_code[idx]
    uniq_runs = np.unique(idx_run_codes)
    rng.shuffle(uniq_runs)
    n_val = max(1, int(len(uniq_runs) * val_ratio))
    val_runs = set(uniq_runs[:n_val].tolist())
    is_val = np.isin(idx_run_codes, np.fromiter(val_runs, dtype=np.int32))

    train_idx = idx[~is_val]
    val_idx   = idx[is_val]

    train_ds = Subset(ds_all, train_idx)
    val_ds   = Subset(ds_all, val_idx)

    train_loader = GeoDataLoader(train_ds, batch_size=batch_size, shuffle=True,  num_workers=num_workers)
    val_loader   = GeoDataLoader(val_ds,   batch_size=batch_size, shuffle=False, num_workers=num_workers)
    return train_loader, val_loader, keep_run_ids, train_idx, val_idx
    
# train_loader, val_loader = make_prb_graph_loaders_lazy(
#     kind="short",
#     batch_size=32,
#     val_ratio=0.2,
#     label_mode="binary",
#     restrict_sources=None,
#     cache_max_runs=1,
#     num_workers=0,
# )

ds_all = PRBWindowDataset(kind="short", label_mode="binary", cache_max_runs=256)
train_loader, val_loader, keep_run_ids, train_idx, val_idx = make_baseline_loaders_from_ds(
    ds_all,
    batch_size=256,
    max_windows_total=300_000,
    keep_attack_runs=2,
    keep_benign_runs=2,
)
print("Using runs:", keep_run_ids)
print("Train windows:", len(train_idx), "Val windows:", len(val_idx))

sample_batch = next(iter(train_loader))
D_NODE = sample_batch.x.size(-1)
print("Detected D_NODE:", D_NODE)  

Using runs: ['040933', '171614', '000600', '001708']
Train windows: 227881 Val windows: 72119
Detected D_NODE: 7


### Base Model Architecture

#### PRB-GraphSAGE Architecture
This code defines `PRBGraph`, a configurable PyTorch Geometric graph neural network for classifying UE-level contention graphs in a PRB starvation detection task. The model builds a GNN backbone with a user-specified number of layers (`num_layers`), hidden dimension, and convolution type (`SAGEConv`, `GCNConv`, `GraphConv`, or `GATv2Conv`), optionally followed by batch normalization and dropout after each layer. In the forward pass, node features `x` are iteratively updated using the chosen conv layers over edges `edge_index`, then aggregated into graph-level embeddings via a selectable global pooling strategy (`mean`, `max`, or concatenated `mean+max`). This pooled representation is passed through an MLP head whose depth and hidden size are also configurable, producing either a scalar logit per graph for binary classification (`num_classes == 1`) or a multi-dimensional logit vector for multi-class problems. The interface is designed to be hyperparameter-sweep friendly, with `edge_attr` included in the signature as a placeholder for future edge-aware enhancements, while the current implementation focuses on node features and graph-level pooling.

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import (
    SAGEConv,
    GCNConv,
    GraphConv,
    GATv2Conv,
    global_mean_pool,
    global_max_pool,
)


class PRBGraphSAGE(nn.Module):
    """
    PRB-GraphSAGE: UE-level contention graph classifier for PRB starvation detection.

    This model is designed to be easy to sweep over hyperparameters:
      - in_dim:         node feature dimension
      - hidden_dim:     hidden channel width
      - num_layers:     number of GNN layers (>=1)
      - conv_type:      GNN layer type: {"sage", "gcn", "graph", "gat"}
      - aggr:           global pooling: {"mean", "max", "mean+max"}
      - mlp_hidden_dim: hidden size of the final MLP head
      - mlp_layers:     number of linear layers in the MLP head (>=1)
      - dropout:        dropout probability applied after each conv & MLP layer
      - num_classes:    1 for binary logit, >1 for multi-class logits

    Forward signature:
        logits = model(x, edge_index, batch, edge_attr=None)

      - x:          [N, in_dim] node features
      - edge_index: [2, E] COO edge indices
      - batch:      [N] graph IDs for global pooling
      - edge_attr:  [E, d_edge] (currently ignored; placeholder for future use)
    """

    def __init__(
        self,
        in_dim: int,
        hidden_dim: int = 64,
        num_layers: int = 2,
        conv_type: str = "sage",
        aggr: str = "mean",
        mlp_hidden_dim: int = 64,
        mlp_layers: int = 2,
        dropout: float = 0.1,
        num_classes: int = 1,
        use_batchnorm: bool = True,
    ):
        super().__init__()

        assert num_layers >= 1, "num_layers must be >= 1"
        assert mlp_layers >= 1, "mlp_layers must be >= 1"
        assert conv_type in {"sage", "gcn", "graph", "gat"}, f"Unknown conv_type: {conv_type}"
        assert aggr in {"mean", "max", "mean+max"}, f"Unknown aggr: {aggr}"
        self.aggr = aggr
        self.dropout = float(dropout)
        self.num_classes = int(num_classes)
        self.use_batchnorm = bool(use_batchnorm)

        # ---- GNN backbone ----
        self.convs = nn.ModuleList()
        self.bns = nn.ModuleList() if use_batchnorm else None

        # input layer
        self.convs.append(self._make_conv(conv_type, in_dim, hidden_dim))
        if use_batchnorm:
            self.bns.append(nn.BatchNorm1d(hidden_dim))

        # hidden layers
        for _ in range(num_layers - 1):
            self.convs.append(self._make_conv(conv_type, hidden_dim, hidden_dim))
            if use_batchnorm:
                self.bns.append(nn.BatchNorm1d(hidden_dim))

        # ---- Readout (global pooling) ----
        # mean+max concatenation doubles the channel dimension
        readout_dim = hidden_dim if aggr in {"mean", "max"} else hidden_dim * 2

        # ---- MLP classifier head ----
        mlp_layers_list = []
        in_mlp = readout_dim
        for li in range(mlp_layers):
            out_mlp = mlp_hidden_dim if li < mlp_layers - 1 else num_classes
            mlp_layers_list.append(nn.Linear(in_mlp, out_mlp))
            if li < mlp_layers - 1:
                mlp_layers_list.append(nn.SiLU())
                if dropout > 0:
                    mlp_layers_list.append(nn.Dropout(dropout))
            in_mlp = out_mlp

        self.mlp = nn.Sequential(*mlp_layers_list)

    @staticmethod
    def _make_conv(conv_type: str, in_dim: int, out_dim: int):
        """Factory for different conv types to make hyperparameter sweeps easy."""
        if conv_type == "sage":
            return SAGEConv(in_dim, out_dim)
        if conv_type == "gcn":
            return GCNConv(in_dim, out_dim)
        if conv_type == "graph":
            return GraphConv(in_dim, out_dim)
        if conv_type == "gat":
            return GATv2Conv(in_dim, out_dim, heads=1, concat=False)
        raise ValueError(f"Unknown conv_type: {conv_type}")

    def forward(self, x, edge_index, batch, edge_attr=None):
        """
        x:          [N, in_dim]
        edge_index: [2, E]
        batch:      [N] graph IDs
        edge_attr:  [E, d_edge] (ignored for now; reserved for future edge-aware layers)
        """
        # ---- GNN backbone ----
        for i, conv in enumerate(self.convs):
            x = conv(x, edge_index)
            x = F.silu(x)
            if self.use_batchnorm:
                x = self.bns[i](x)
            if self.dropout > 0:
                x = F.dropout(x, p=self.dropout, training=self.training)

        # ---- Global pooling ----
        if self.aggr == "mean":
            g = global_mean_pool(x, batch)
        elif self.aggr == "max":
            g = global_max_pool(x, batch)
        else:  # "mean+max"
            g_mean = global_mean_pool(x, batch)
            g_max = global_max_pool(x, batch)
            g = torch.cat([g_mean, g_max], dim=-1)

        # ---- MLP head ----
        logits = self.mlp(g)  # [num_graphs, num_classes]

        # For binary classification, most training loops will want shape [num_graphs]
        if self.num_classes == 1:
            logits = logits.squeeze(-1)

        return logits

### Base Model Initialization

#### Hyperparameters

This code sets up initialization, configuration, and lightweight hyperparameter sweeping for a PRB-Graph GNN used in PRB starvation detection. It begins by seeding Python, NumPy, and PyTorch RNGs for reproducibility, choosing a CUDA or CPU device, and defining `D_NODE` as the node feature dimension (with a placeholder warning until real graph data is available). A `GNN_HYP` dictionary gathers all key hyperparameters in one place—training settings (batch size, epochs, learning rate and grid, weight decay), backbone architecture (hidden size, number of layers, dropout, conv type, pooling strategy, batchnorm), MLP head configuration, output dimension (binary vs multi-class), early-stopping criteria, and scheduler parameters (warmup and total steps). A helper `compute_pos_weight_from_labels` computes class weights for `BCEWithLogitsLoss` from imbalanced binary labels, with a default `pos_weight` of 1.0 until real labels are known. The `make_gnn_model` factory instantiates a `PRBGraphSAGE` model from a hyperparameter dict and moves it to the chosen device, while `count_params` reports trainable parameter count. Optimizer and scheduler builders create an AdamW optimizer and a LambdaLR scheduler implementing linear warmup followed by cosine decay over `total_steps`. The script then instantiates a default model, optimizer, scheduler, and a binary `BCEWithLogitsLoss` using the current `pos_weight`, printing summaries. Finally, `iter_gnn_configs` defines a simple grid-search generator over selected hyperparameters (learning rate, hidden size, number of layers, dropout, aggregation, and conv type), and the code demonstrates its use by instantiating and printing the first few candidate configurations along with their parameter counts, providing a ready-made loop for hyperparameter sweeps.

In [4]:
# ==== CELL: GNN Model + Hyperparameter Initialization ====
import os, math, json, random
from typing import Dict, Iterable

import numpy as np
import torch
import torch.nn as nn

# ---- Reproducibility & device ----
SEED = 2025

def seed_all(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = False
    torch.backends.cudnn.benchmark = True

seed_all(SEED)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", DEVICE)

# ---- GNN Hyperparameters (with small grids for sweeps) ----
GNN_HYP: Dict = {
    # ---- Training-level hparams ----
    "batch_size": 32, # Any changes must also be copied to the data loader              
    "epochs": 40,                  
    "lr": 3e-4,                    
    "lr_grid": [2e-4, 3e-4, 5e-4], 
    "weight_decay": 1e-4,

    # ---- Backbone architecture ----
    "hidden_dim": 64,
    "num_layers": 2,
    "dropout": 0.1,
    "conv_type": "sage",           
    "aggr": "mean",                
    "use_batchnorm": True,

    # Grids for a compact but meaningful sweep
    "hidden_dim_grid": [32, 64],           
    "num_layers_grid": [1, 2, 3],          
    "dropout_grid": [0.0, 0.1, 0.3],       
    "aggr_grid": ["mean", "mean+max"],     
    "conv_type_grid": ["sage", "gcn"],     

    # ---- MLP head ----
    "mlp_hidden_dim": 64,
    "mlp_layers": 2,

    # ---- Output ----
    # 1 = binary logit (BCEWithLogits); >1 = multi-class (CrossEntropy)
    "num_classes": 1,

    # ---- Early-stop config (used by your training loop) ----
    "early_stop": {
        "monitor": "f1",            
        "mode": "max",              
        "patience": 5,              
        "min_delta": 1e-3,          
    },

    # ---- Scheduler placeholders (updated once loaders are known) ----
    "warmup_steps": 200,
    "total_steps": 10000,
}

print("GNN Hyperparameters:", json.dumps(GNN_HYP, indent=2))


# ---- Optional class weights for binary BCEWithLogitsLoss ----
# You can adapt this to your graph meta if you materialize labels in a DataFrame.
def compute_pos_weight_from_labels(labels: np.ndarray) -> torch.Tensor:
    """
    Compute pos_weight for BCEWithLogitsLoss from a binary label vector.
    pos_weight = (#neg / #pos)
    """
    labels = labels.astype(np.int64).ravel()
    pos = (labels == 1).sum()
    neg = (labels == 0).sum()
    if pos == 0:
        return torch.tensor(1.0, dtype=torch.float32)
    return torch.tensor(float(neg) / float(pos), dtype=torch.float32)

pos_weight = torch.tensor(1.0, dtype=torch.float32)
print("Initial pos_weight (binary):", float(pos_weight))


# ---- PRB-GraphSAGE factory ----
def make_gnn_model(hyp: Dict, in_dim: int) -> PRBGraphSAGE:
    """
    Instantiate a PRBGraphSAGE model from a hyperparameter dict.
    """
    model = PRBGraphSAGE(
        in_dim=in_dim,
        hidden_dim=hyp["hidden_dim"],
        num_layers=hyp["num_layers"],
        conv_type=hyp["conv_type"],      
        aggr=hyp["aggr"],                
        mlp_hidden_dim=hyp["mlp_hidden_dim"],
        mlp_layers=hyp["mlp_layers"],
        dropout=hyp["dropout"],
        num_classes=hyp["num_classes"],
        use_batchnorm=hyp["use_batchnorm"],
    )
    return model.to(DEVICE)



def count_params(m: nn.Module) -> int:
    return sum(p.numel() for p in m.parameters() if p.requires_grad)


# ---- Optimizer & Scheduler helpers ----
def make_gnn_optimizer(model: nn.Module, hyp: Dict) -> torch.optim.Optimizer:
    return torch.optim.AdamW(
        model.parameters(),
        lr=hyp["lr"],
        weight_decay=hyp["weight_decay"],
    )

def make_scheduler(optimizer, warmup_steps: int, total_steps: int):
    def lr_lambda(step):
        if step < warmup_steps:
            return float(step) / float(max(1, warmup_steps))
        progress = float(step - warmup_steps) / float(max(1, total_steps - warmup_steps))
        return 0.5 * (1.0 + math.cos(math.pi * progress))
    return torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)


# ---- Instantiate a default GNN model + optimizer/scheduler ----
gnn_model = make_gnn_model(GNN_HYP, D_NODE)
print(f"GNN params: {count_params(gnn_model):,}")

gnn_optim = make_gnn_optimizer(gnn_model, GNN_HYP)
gnn_scheduler = make_scheduler(gnn_optim, GNN_HYP["warmup_steps"], GNN_HYP["total_steps"])

# Binary classification by default (y ∈ {0,1})
gnn_criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight.to(DEVICE))

print("GNN model, optimizer, scheduler, and loss are initialized.")


# ---- Hyperparameter sweep helper (config generator only) ----
def iter_gnn_configs(base: Dict) -> Iterable[Dict]:
    """
    Simple grid over a few key hyperparameters.
    You can extend/prune this as needed.
    """
    for lr in base["lr_grid"]:
        for hidden in base["hidden_dim_grid"]:
            for num_layers in base["num_layers_grid"]:
                for dropout in base["dropout_grid"]:
                    for aggr in base["aggr_grid"]:
                        for conv_type in base.get("conv_type_grid", [base["conv_type"]]):
                            cfg = dict(base)  # shallow copy
                            cfg["lr"] = lr
                            cfg["hidden_dim"] = hidden
                            cfg["num_layers"] = num_layers
                            cfg["dropout"] = dropout
                            cfg["aggr"] = aggr
                            cfg["conv_type"] = conv_type
                            yield cfg

print("\nInitialization")
for i, cfg in enumerate(iter_gnn_configs(GNN_HYP)):
    if i >= 3:
        break
    m = make_gnn_model(cfg, D_NODE)
    print(
        f"  Config {i}: lr={cfg['lr']}, hidden={cfg['hidden_dim']}, "
        f"layers={cfg['num_layers']}, dropout={cfg['dropout']}, "
        f"aggr={cfg['aggr']}, conv_type={cfg['conv_type']}, "
        f"params={count_params(m):,}"
    )

Device: cuda
GNN Hyperparameters: {
  "batch_size": 32,
  "epochs": 40,
  "lr": 0.0003,
  "lr_grid": [
    0.0002,
    0.0003,
    0.0005
  ],
  "weight_decay": 0.0001,
  "hidden_dim": 64,
  "num_layers": 2,
  "dropout": 0.1,
  "conv_type": "sage",
  "aggr": "mean",
  "use_batchnorm": true,
  "hidden_dim_grid": [
    32,
    64
  ],
  "num_layers_grid": [
    1,
    2,
    3
  ],
  "dropout_grid": [
    0.0,
    0.1,
    0.3
  ],
  "aggr_grid": [
    "mean",
    "mean+max"
  ],
  "conv_type_grid": [
    "sage",
    "gcn"
  ],
  "mlp_hidden_dim": 64,
  "mlp_layers": 2,
  "num_classes": 1,
  "early_stop": {
    "monitor": "f1",
    "mode": "max",
    "patience": 5,
    "min_delta": 0.001
  },
  "warmup_steps": 200,
  "total_steps": 10000
}
Initial pos_weight (binary): 1.0
GNN params: 13,697
GNN model, optimizer, scheduler, and loss are initialized.

Initialization
  Config 0: lr=0.0002, hidden=32, layers=1, dropout=0.0, aggr=mean, conv_type=sage, params=2,721
  Config 1: lr=0.0002, hidde

/home/ryan/ml4g/term_project/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:283: UserWarning: 
    Found GPU0 NVIDIA GB10 which is of cuda capability 12.1.
    Minimum and Maximum cuda capability supported by this version of PyTorch is
    (8.0) - (12.0)
    
  warnings.warn(


### Model Training Loop

#### PRB-Graph

##### Helper Functions

These helpers implement a compact training/evaluation loop for a binary graph classifier and standard performance metrics. compute_binary_metrics takes ground-truth binary labels and predicted probabilities (post-sigmoid), thresholds them into hard predictions, and computes accuracy, precision, recall, and F1 score with small numerical safeguards. run_epoch iterates over a PyTorch Geometric DataLoader for one epoch in either training or evaluation mode, moves each batch to the target device, runs the model forward on graph data (x, edge_index, batch), computes a scalar loss against flattened labels using a given criterion (e.g., BCEWithLogitsLoss), and in training mode performs backpropagation, optimizer steps, and optional scheduler updates. It accumulates loss weighted by batch size to return an average epoch loss, collects all sigmoid probabilities and labels across batches, and finally calls compute_binary_metrics to produce a metrics dictionary augmented with the averaged loss, or returns zeros if no samples were seen.

In [5]:
import copy
from typing import Dict, Tuple, List
import numpy as np
import torch
import torch.nn as nn

def compute_binary_metrics(y_true: np.ndarray,
                           y_prob: np.ndarray,
                           threshold: float = 0.5) -> Dict[str, float]:
    """
    y_true: [N] in {0,1}
    y_prob: [N] in [0,1]  (sigmoid outputs)
    """
    y_true = y_true.astype(np.int64).ravel()
    y_prob = y_prob.astype(np.float32).ravel()
    y_pred = (y_prob >= threshold).astype(np.int64)

    tp = np.sum((y_pred == 1) & (y_true == 1))
    tn = np.sum((y_pred == 0) & (y_true == 0))
    fp = np.sum((y_pred == 1) & (y_true == 0))
    fn = np.sum((y_pred == 0) & (y_true == 1))

    eps = 1e-9
    precision = tp / (tp + fp + eps)
    recall    = tp / (tp + fn + eps)
    f1        = 2 * precision * recall / (precision + recall + eps)
    acc       = (tp + tn) / max(1, len(y_true))

    return {
        "acc": float(acc),
        "precision": float(precision),
        "recall": float(recall),
        "f1": float(f1),
    }

def run_epoch(model: nn.Module,
              loader,
              optimizer: torch.optim.Optimizer,
              criterion: nn.Module,
              device: torch.device,
              scheduler=None,
              train: bool = True) -> Tuple[float, Dict[str, float]]:
    """
    Runs one epoch over loader.
    Returns (avg_loss, metrics_dict).
    """
    if train:
        model.train()
    else:
        model.eval()

    all_probs: List[float] = []
    all_labels: List[int] = []
    total_loss = 0.0
    total_samples = 0

    for batch in loader:
        batch = batch.to(device)
        # y: [num_graphs] or [num_graphs, 1]
        y = batch.y.view(-1).float()

        with torch.set_grad_enabled(train):
            logits = model(batch.x, batch.edge_index, batch.batch)  # [num_graphs] or [num_graphs,1]
            logits = logits.view_as(y)
            loss = criterion(logits, y)

            if train:
                optimizer.zero_grad(set_to_none=True)
                loss.backward()
                optimizer.step()
                if scheduler is not None:
                    scheduler.step()

        bs = y.size(0)
        total_loss += float(loss.item()) * bs
        total_samples += bs

        probs = torch.sigmoid(logits.detach()).cpu().numpy()
        labels = y.detach().cpu().numpy()
        all_probs.append(probs)
        all_labels.append(labels)

    if total_samples == 0:
        return 0.0, {"acc": 0.0, "precision": 0.0, "recall": 0.0, "f1": 0.0}

    all_probs = np.concatenate(all_probs, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)
    avg_loss = total_loss / total_samples
    metrics = compute_binary_metrics(all_labels, all_probs, threshold=0.5)
    metrics["loss"] = float(avg_loss)
    return avg_loss, metrics

##### Training Loop (with Early Stop)

This code implements an early-stopping mechanism and a high-level training loop for a single GNN configuration. The `EarlyStopper` class tracks a chosen validation metric (e.g., F1) over epochs, deciding whether to stop training when the metric fails to improve by at least `min_delta` for a specified number of epochs (`patience`), and it keeps a deep copy of the best model state encountered. The `train_gnn_model` function wires everything together for one run: it constructs a GNN using `make_gnn_model`, sets up an AdamW optimizer and cosine–decay scheduler via `make_gnn_optimizer` and `make_scheduler`, and uses a shared loss (`gnn_criterion`, typically `BCEWithLogitsLoss` for binary classification). For each epoch, it calls `run_epoch` on both the training and validation loaders, capturing losses and metrics, logs progress with train/val loss and F1, appends snapshots into a `history` dict, and feeds validation metrics into `EarlyStopper` to determine if early stopping should trigger. When training ends—either by exhausting the epoch budget or by early stop—the best-performing weights (if any) are restored onto the model, and a result dictionary is returned containing the trained model, metric history, the best monitored metric value, its name, and the corresponding `state_dict` for downstream evaluation or checkpointing.


In [6]:
class EarlyStopper:
    def __init__(self, monitor: str = "f1", mode: str = "max",
                 patience: int = 5, min_delta: float = 1e-3):
        self.monitor = monitor
        self.mode = mode
        self.patience = patience
        self.min_delta = min_delta

        if mode == "max":
            self.best = -float("inf")
        elif mode == "min":
            self.best = float("inf")
        else:
            raise ValueError("mode must be 'max' or 'min'")

        self.num_bad = 0
        self.best_state = None

    def step(self, metrics: Dict[str, float], model: nn.Module) -> bool:
        """
        Returns True if we should stop (patience exhausted).
        """
        current = metrics.get(self.monitor, None)
        if current is None:
            return False

        improved = False
        if self.mode == "max":
            if current > self.best + self.min_delta:
                improved = True
        else:  # 'min'
            if current < self.best - self.min_delta:
                improved = True

        if improved:
            self.best = current
            self.num_bad = 0
            self.best_state = copy.deepcopy(model.state_dict())
        else:
            self.num_bad += 1

        return self.num_bad >= self.patience


def train_gnn_model(
    hyp: Dict,
    train_loader,
    val_loader,
    in_dim: int,
    device: torch.device = DEVICE,
) -> Dict:
    """
    High-level training loop for a single GNN config.
    Returns a dict with best metrics, history, and best_state_dict.
    """

    model = make_gnn_model(hyp, in_dim=in_dim)
    optimizer = make_gnn_optimizer(model, hyp)
    scheduler = make_scheduler(optimizer, hyp["warmup_steps"], hyp["total_steps"])

    criterion = gnn_criterion 

    es_cfg = hyp["early_stop"]
    early_stopper = EarlyStopper(
        monitor=es_cfg.get("monitor", "f1"),
        mode=es_cfg.get("mode", "max"),
        patience=es_cfg.get("patience", 5),
        min_delta=es_cfg.get("min_delta", 1e-3),
    )

    history = {"train": [], "val": []}

    for epoch in range(1, hyp["epochs"] + 1):
        train_loss, train_metrics = run_epoch(
            model, train_loader, optimizer, criterion, device, scheduler, train=True
        )
        val_loss, val_metrics = run_epoch(
            model, val_loader, optimizer, criterion, device, scheduler=None, train=False
        )

        history["train"].append(train_metrics)
        history["val"].append(val_metrics)

        print(
            f"[Epoch {epoch:03d}] "
            f"train_loss={train_metrics['loss']:.4f}, train_f1={train_metrics['f1']:.3f} | "
            f"val_loss={val_metrics['loss']:.4f}, val_f1={val_metrics['f1']:.3f}"
        )

        stop = early_stopper.step(val_metrics, model)
        if stop:
            print(f"Early stopping triggered at epoch {epoch}. Best {early_stopper.monitor} = {early_stopper.best:.4f}")
            break

    # Restore best weights if we got any improvement
    if early_stopper.best_state is not None:
        model.load_state_dict(early_stopper.best_state)

    result = {
        "model": model,
        "history": history,
        "best_metric": early_stopper.best,
        "best_monitor": early_stopper.monitor,
        "best_state_dict": early_stopper.best_state,
    }
    return result

##### Initiate Training

This code performs a simple grid search over GNN hyperparameters and tracks the best-performing configuration based on validation metrics. It iterates through each candidate config generated by `iter_gnn_configs(GNN_HYP)`, prints a short summary of that config (convolution type, hidden dimension, number of layers, dropout, aggregation, and learning rate), and trains a model with `train_gnn_model` on the given train/validation loaders and node feature dimension. The result `out` from each run includes the best monitored metric (e.g., validation F1) for that configuration; the loop keeps `best_overall` and `best_cfg` updated whenever a configuration surpasses the current best metric. After all configurations have been evaluated, it prints the best hyperparameter dictionary and the corresponding best validation score, effectively selecting the strongest model configuration discovered during the sweep.

In [7]:
best_overall = None
best_cfg = None

for cfg in iter_gnn_configs(GNN_HYP):
    print("\n=== Config:", cfg["conv_type"], cfg["hidden_dim"], cfg["num_layers"],
          cfg["dropout"], cfg["aggr"], "lr", cfg["lr"], "===")
    out = train_gnn_model(cfg, train_loader, val_loader, in_dim=D_NODE, device=DEVICE)

    if best_overall is None or out["best_metric"] > best_overall["best_metric"]:
        best_overall = out
        best_cfg = cfg

print("\nBest config:", best_cfg)
print("Best validation", best_overall["best_monitor"], "=", best_overall["best_metric"])


=== Config: sage 32 1 0.0 mean lr 0.0002 ===
[Epoch 001] train_loss=0.2898, train_f1=0.119 | val_loss=1.3020, val_f1=0.091
[Epoch 002] train_loss=0.1074, train_f1=0.116 | val_loss=3.0173, val_f1=0.010
[Epoch 003] train_loss=0.0887, train_f1=0.083 | val_loss=5.2647, val_f1=0.000
[Epoch 004] train_loss=0.0839, train_f1=0.011 | val_loss=6.5238, val_f1=0.000
[Epoch 005] train_loss=0.0823, train_f1=0.094 | val_loss=6.8929, val_f1=0.000
[Epoch 006] train_loss=0.0815, train_f1=0.146 | val_loss=6.9423, val_f1=0.000
Early stopping triggered at epoch 6. Best f1 = 0.0911

=== Config: gcn 32 1 0.0 mean lr 0.0002 ===
[Epoch 001] train_loss=0.3060, train_f1=0.128 | val_loss=1.6557, val_f1=0.118
[Epoch 002] train_loss=0.1038, train_f1=0.192 | val_loss=4.6488, val_f1=0.009
[Epoch 003] train_loss=0.0918, train_f1=0.246 | val_loss=7.3088, val_f1=0.117
[Epoch 004] train_loss=0.0870, train_f1=0.235 | val_loss=9.6750, val_f1=0.000
[Epoch 005] train_loss=0.0843, train_f1=0.195 | val_loss=10.8791, val_f1=0.

/home/ryan/ml4g/term_project/.venv/lib/python3.12/site-packages/torch_geometric/utils/_scatter.py:91: UserWarning: The usage of `scatter(reduce='max')` can be accelerated via the 'torch-scatter' package, but it was not found
  warnings.warn(


[Epoch 001] train_loss=0.2485, train_f1=0.044 | val_loss=1.5313, val_f1=0.010
[Epoch 002] train_loss=0.1006, train_f1=0.063 | val_loss=3.2868, val_f1=0.009
[Epoch 003] train_loss=0.0880, train_f1=0.056 | val_loss=5.0401, val_f1=0.000
[Epoch 004] train_loss=0.0833, train_f1=0.149 | val_loss=5.6700, val_f1=0.122
[Epoch 005] train_loss=0.0814, train_f1=0.231 | val_loss=6.1568, val_f1=0.000
[Epoch 006] train_loss=0.0808, train_f1=0.253 | val_loss=5.7140, val_f1=0.122
[Epoch 007] train_loss=0.0803, train_f1=0.284 | val_loss=5.7236, val_f1=0.000
[Epoch 008] train_loss=0.0801, train_f1=0.291 | val_loss=5.5936, val_f1=0.122
[Epoch 009] train_loss=0.0801, train_f1=0.284 | val_loss=5.3249, val_f1=0.122
Early stopping triggered at epoch 9. Best f1 = 0.1222

=== Config: gcn 32 1 0.0 mean+max lr 0.0002 ===
[Epoch 001] train_loss=0.2324, train_f1=0.146 | val_loss=3.7689, val_f1=0.009
[Epoch 002] train_loss=0.0933, train_f1=0.270 | val_loss=8.8689, val_f1=0.117
[Epoch 003] train_loss=0.0857, train_f1

### Model Evaluation

#### PRB-Graph

##### Helper Functions

This code provides evaluation utilities for a trained PRB-GraphSAGE model, computing metrics and confusion statistics on a given dataset. The `evaluate_gnn_model` function runs in no-grad mode, sets the model to eval, and iterates over a PyTorch Geometric loader, moving each batch to the target device, forwarding node features and graph structure through the model to obtain logits, and computing a loss with the same `gnn_criterion` used during training. It accumulates batch-weighted loss, collects sigmoid-transformed probabilities and true labels, and at the end concatenates them to compute average loss and standard binary classification metrics (accuracy, precision, recall, F1) via the existing `compute_binary_metrics` helper, returning both the metrics dictionary and the full `y_true` and `y_prob` arrays. The `summarize_eval_split` helper prints these metrics in a compact, human-readable line for a named split (e.g., “train”, “val”, “test”). Finally, `confusion_from_probs` turns true labels and predicted probabilities into a small 2×2 confusion matrix (tp, tn, fp, fn) using a configurable threshold, making it easy to inspect the model’s error profile on any evaluation split.

In [8]:
# ==== CELL: PRB-GraphSAGE Evaluation (Best Model) ====
import numpy as np
import torch
from typing import Dict, Tuple


@torch.no_grad()
def evaluate_gnn_model(
    model: torch.nn.Module,
    loader,
    device: torch.device,
    threshold: float = 0.5,
) -> Tuple[Dict[str, float], np.ndarray, np.ndarray]:
    """
    Evaluate a trained PRB-GraphSAGE model on a given loader.

    Returns:
        metrics: dict with loss, acc, precision, recall, f1
        y_true:  np.ndarray of true labels (0/1)
        y_prob:  np.ndarray of predicted probabilities p(y=1)
    """
    model.eval()
    all_probs = []
    all_labels = []
    total_loss = 0.0
    total_samples = 0

    for batch in loader:
        batch = batch.to(device)
        y = batch.y.view(-1).float()            # [B]

        logits = model(batch.x, batch.edge_index, batch.batch)  # [B] or [B,1]
        logits = logits.view_as(y)
        loss = gnn_criterion(logits, y)

        bs = y.size(0)
        total_loss += float(loss.item()) * bs
        total_samples += bs

        probs = torch.sigmoid(logits).cpu().numpy()
        labels = y.cpu().numpy()
        all_probs.append(probs)
        all_labels.append(labels)

    if total_samples == 0:
        metrics = {"loss": 0.0, "acc": 0.0, "precision": 0.0, "recall": 0.0, "f1": 0.0}
        return metrics, np.array([]), np.array([])

    all_probs = np.concatenate(all_probs, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)
    avg_loss = total_loss / total_samples

    # Reuse the helper you already defined earlier
    metrics = compute_binary_metrics(all_labels, all_probs, threshold=threshold)
    metrics["loss"] = float(avg_loss)
    return metrics, all_labels, all_probs


def summarize_eval_split(name: str, metrics: Dict[str, float]):
    print(
        f"[{name}] "
        f"loss={metrics['loss']:.4f}  "
        f"acc={metrics['acc']:.3f}  "
        f"precision={metrics['precision']:.3f}  "
        f"recall={metrics['recall']:.3f}  "
        f"f1={metrics['f1']:.3f}"
    )


def confusion_from_probs(y_true: np.ndarray, y_prob: np.ndarray, threshold: float = 0.5) -> Dict[str, int]:
    """
    Tiny 2x2 confusion matrix helper for reporting.
    """
    y_true = y_true.astype(np.int64).ravel()
    y_pred = (y_prob >= threshold).astype(np.int64).ravel()

    tp = int(((y_pred == 1) & (y_true == 1)).sum())
    tn = int(((y_pred == 0) & (y_true == 0)).sum())
    fp = int(((y_pred == 1) & (y_true == 0)).sum())
    fn = int(((y_pred == 0) & (y_true == 1)).sum())

    return {"tp": tp, "tn": tn, "fp": fp, "fn": fn}

##### Evaluation

This code evaluates the best-performing PRB-Graph model found during the hyperparameter sweep on both the training and validation splits to assess fit and generalization. It first retrieves the model with its restored best weights from `best_overall`, moves it to the active device, switches it to evaluation mode, and prints the corresponding best hyperparameter configuration and monitored metric (e.g., validation F1). It then calls `evaluate_gnn_model` on the training DataLoader to compute loss and binary metrics, uses `summarize_eval_split` to print a concise summary line, and derives a 2×2 confusion matrix with `confusion_from_probs` to inspect true/false positives and negatives. The same process is repeated on the validation DataLoader, with the validation metrics and confusion matrix serving as the primary performance numbers to report in experiments or papers.

In [9]:
# Grab the best model (weights are already restored in train_gnn_model)
best_model = best_overall["model"].to(DEVICE)
best_model.eval()

print("Best config:", best_cfg)
print(f"Best monitored metric ({best_overall['best_monitor']}): {best_overall['best_metric']:.4f}\n")

# Evaluate on training split (to see overfit / underfit)
train_metrics, train_y, train_p = evaluate_gnn_model(best_model, train_loader, DEVICE, threshold=0.5)
summarize_eval_split("train", train_metrics)
train_cm = confusion_from_probs(train_y, train_p, threshold=0.5)
print("  train confusion:", train_cm)

# Evaluate on validation split (primary number for the report)
val_metrics, val_y, val_p = evaluate_gnn_model(best_model, val_loader, DEVICE, threshold=0.5)
summarize_eval_split("val", val_metrics)
val_cm = confusion_from_probs(val_y, val_p, threshold=0.5)
print("  val confusion:", val_cm)

Best config: {'batch_size': 32, 'epochs': 40, 'lr': 0.0002, 'lr_grid': [0.0002, 0.0003, 0.0005], 'weight_decay': 0.0001, 'hidden_dim': 32, 'num_layers': 2, 'dropout': 0.3, 'conv_type': 'gcn', 'aggr': 'mean', 'use_batchnorm': True, 'hidden_dim_grid': [32, 64], 'num_layers_grid': [1, 2, 3], 'dropout_grid': [0.0, 0.1, 0.3], 'aggr_grid': ['mean', 'mean+max'], 'conv_type_grid': ['sage', 'gcn'], 'mlp_hidden_dim': 64, 'mlp_layers': 2, 'num_classes': 1, 'early_stop': {'monitor': 'f1', 'mode': 'max', 'patience': 5, 'min_delta': 0.001}, 'warmup_steps': 200, 'total_steps': 10000}
Best monitored metric (f1): 0.2265

[train] loss=0.0857  acc=0.940  precision=0.452  recall=0.979  f1=0.618
  train confusion: {'tp': 11133, 'tn': 202990, 'fp': 13522, 'fn': 236}
[val] loss=4.5763  acc=0.385  precision=0.991  recall=0.128  f1=0.227
  val confusion: {'tp': 6491, 'tn': 21305, 'fp': 60, 'fn': 44263}


## Export Model

In [11]:
import os
import torch

EXPORT_DIR = "models"
os.makedirs(EXPORT_DIR, exist_ok=True)
EXPORT_PATH = os.path.join(EXPORT_DIR, "best_gnn_model.pt")

best_model = best_overall["model"].to(DEVICE)
best_model.eval()

print("Best config:", best_cfg)
print(f"Best monitored metric ({best_overall['best_monitor']}): {best_overall['best_metric']:.4f}\n")

train_metrics, train_y, train_p = evaluate_gnn_model(best_model, train_loader, DEVICE, threshold=0.5)
summarize_eval_split("train", train_metrics)
train_cm = confusion_from_probs(train_y, train_p, threshold=0.5)
print("  train confusion:", train_cm)

val_metrics, val_y, val_p = evaluate_gnn_model(best_model, val_loader, DEVICE, threshold=0.5)
summarize_eval_split("val", val_metrics)
val_cm = confusion_from_probs(val_y, val_p, threshold=0.5)
print("  val confusion:", val_cm)

checkpoint = {
    "model_state_dict": best_model.state_dict(),
    "config": best_cfg,                         # whatever you used to build the model
    "best_monitor": best_overall["best_monitor"],
    "best_metric": best_overall["best_metric"],
}

torch.save(checkpoint, EXPORT_PATH)
print(f"\nSaved best model checkpoint to: {EXPORT_PATH}")

Best config: {'batch_size': 32, 'epochs': 40, 'lr': 0.0002, 'lr_grid': [0.0002, 0.0003, 0.0005], 'weight_decay': 0.0001, 'hidden_dim': 32, 'num_layers': 2, 'dropout': 0.3, 'conv_type': 'gcn', 'aggr': 'mean', 'use_batchnorm': True, 'hidden_dim_grid': [32, 64], 'num_layers_grid': [1, 2, 3], 'dropout_grid': [0.0, 0.1, 0.3], 'aggr_grid': ['mean', 'mean+max'], 'conv_type_grid': ['sage', 'gcn'], 'mlp_hidden_dim': 64, 'mlp_layers': 2, 'num_classes': 1, 'early_stop': {'monitor': 'f1', 'mode': 'max', 'patience': 5, 'min_delta': 0.001}, 'warmup_steps': 200, 'total_steps': 10000}
Best monitored metric (f1): 0.2265

[train] loss=0.0857  acc=0.940  precision=0.452  recall=0.979  f1=0.618
  train confusion: {'tp': 11133, 'tn': 202990, 'fp': 13522, 'fn': 236}
[val] loss=4.5763  acc=0.385  precision=0.991  recall=0.128  f1=0.227
  val confusion: {'tp': 6491, 'tn': 21305, 'fp': 60, 'fn': 44263}

Saved best model checkpoint to: models/best_gnn_model.pt


### Visualization Code

In [17]:
# ==== CELL: Visualization – PR Curve, Confusion Matrix, and Gradient-Based Attributions ====
import os
import numpy as np
import torch
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, average_precision_score, confusion_matrix

# -----------------------------------------------------------------------------
# 1. Load best model from checkpoint
# -----------------------------------------------------------------------------
FIG_DIR = "figures"
os.makedirs(FIG_DIR, exist_ok=True)

CKPT_PATH = "models/best_gnn_model.pt"
checkpoint = torch.load(CKPT_PATH, map_location=DEVICE)

best_cfg = checkpoint["config"]
print("Loaded best config from checkpoint:", best_cfg)
print("Best validation metric stored:", checkpoint.get("best_monitor"), "=", checkpoint.get("best_metric"))

# Rebuild model architecture and load weights
viz_model = make_gnn_model(best_cfg, D_NODE)
viz_model.load_state_dict(checkpoint["model_state_dict"])
viz_model.to(DEVICE)
viz_model.eval()

# -----------------------------------------------------------------------------
# 2. Precision–Recall curve + confusion matrix on held-out validation data
# -----------------------------------------------------------------------------
val_metrics, y_true_val, y_prob_val = evaluate_gnn_model(
    viz_model,
    val_loader,
    DEVICE,
    threshold=0.5,  # threshold here is only for metrics; we'll sweep separately
)

print("\n[VAL] Metrics at threshold 0.5 used during training:")
print(val_metrics)

# Compute PR curve
precision, recall, pr_thresholds = precision_recall_curve(y_true_val, y_prob_val)
ap = average_precision_score(y_true_val, y_prob_val)

# Find threshold that maximizes F1 over the PR curve points
# Note: precision_recall_curve returns len(thresholds)+1 points; F1 is computed at those points.
eps = 1e-9
f1_points = 2 * precision * recall / (precision + recall + eps)
best_idx = int(np.nanargmax(f1_points))
best_f1_est = float(f1_points[best_idx])

if best_idx == 0:
    # first PR point has no associated threshold; fall back to 0.5
    best_thr = 0.5
else:
    best_thr = float(pr_thresholds[best_idx - 1])

print(f"\n[VAL] Best F1 over PR curve ≈ {best_f1_est:.3f} at threshold ≈ {best_thr:.4f}")

# Confusion matrix at best_thr
y_pred_best = (y_prob_val >= best_thr).astype(int)
cm = confusion_matrix(y_true_val, y_pred_best, labels=[0, 1])
tn, fp, fn, tp = cm.ravel()

print("\n[VAL] Confusion matrix at threshold ≈ {:.4f}:".format(best_thr))
print("TN:", tn, "FP:", fp, "FN:", fn, "TP:", tp)

# Plot PR curve + confusion matrix
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# --- PR curve ---
ax_pr = axes[0]
ax_pr.plot(recall, precision, drawstyle="steps-post")
ax_pr.set_xlabel("Recall")
ax_pr.set_ylabel("Precision")
ax_pr.set_title(f"Validation Precision–Recall (AP = {ap:.3f})")
ax_pr.set_xlim([0.0, 1.0])
ax_pr.set_ylim([0.0, 1.05])

# Mark best F1 point
ax_pr.scatter(recall[best_idx], precision[best_idx], color="red", label=f"Best F1≈{best_f1_est:.3f}")
ax_pr.legend(loc="lower left")

# --- Confusion matrix ---
ax_cm = axes[1]
im = ax_cm.imshow(cm, cmap="Blues")
ax_cm.set_xticks([0, 1], labels=["Pred 0 (benign)", "Pred 1 (attack)"], rotation=15, ha="right")
ax_cm.set_yticks([0, 1], labels=["True 0 (benign)", "True 1 (attack)"])
ax_cm.set_title(f"Confusion Matrix @ threshold ≈ {best_thr:.4f}")

# Annotate each cell
for i in range(2):
    for j in range(2):
        ax_cm.text(
            j,
            i,
            str(cm[i, j]),
            ha="center",
            va="center",
            color="white" if cm[i, j] > cm.max() / 2 else "black",
            fontsize=10,
        )

fig.colorbar(im, ax=ax_cm, fraction=0.046, pad=0.04)
plt.tight_layout()
pr_cm_path = os.path.join(FIG_DIR, "fig_pr_curve_and_confusion_val.png")
plt.savefig(pr_cm_path, dpi=300, bbox_inches="tight")
plt.close(fig)
print(f"\nSaved PR curve + confusion matrix figure to: {pr_cm_path}")

# -----------------------------------------------------------------------------
# 3. Gradient-based node×feature attributions (GNNExplainer-style)
# -----------------------------------------------------------------------------
def find_example_by_label(dataset, target_label: int):
    """
    Return the first graph Data object in `dataset` whose y == target_label and has nodes.
    """
    for idx in range(len(dataset)):
        d = dataset[idx]
        if d.x is None or d.x.numel() == 0:
            continue
        if d.edge_index is None:
            continue
        if int(d.y.item()) == target_label:
            return d, idx
    return None, None

def compute_node_feature_importance(model: torch.nn.Module, data) -> np.ndarray:
    """
    Gradient×input node feature importance:
      importance[node, feature] = |grad * x|
    """
    model.eval()

    # Prepare inputs with gradient tracking only on x
    x = data.x.to(DEVICE).clone().detach()
    x.requires_grad_(True)
    edge_index = data.edge_index.to(DEVICE)
    batch = torch.zeros(x.size(0), dtype=torch.long, device=DEVICE)  # single graph

    model.zero_grad(set_to_none=True)

    logits = model(x, edge_index, batch)  # scalar logit for binary classification
    logit = logits.view(-1)[0]
    logit.backward()

    grad = x.grad  # same shape as x
    # Detach *after* grad×input
    importance = (grad * x).abs().detach().cpu().numpy()  # [N, F]
    return importance

# Feature names must match build_node_features() ordering:
feature_names = [
    "mean_share",
    "min_share",
    "max_share",
    "std_share",
    "active_frac",
    "zero_frac",
    "longest_zero_run",
]

# Find representative benign (0) and attack (1) graphs
benign_data, benign_idx = find_example_by_label(ds_all, target_label=0)
attack_data, attack_idx = find_example_by_label(ds_all, target_label=1)

if benign_data is None or attack_data is None:
    print("\n[WARN] Could not find both benign and attack examples in ds_all; "
          "skipping attribution heatmaps.")
else:
    print(f"\nUsing benign example at ds_all index {benign_idx}, "
          f"attack example at ds_all index {attack_idx} for attributions.")

    benign_imp = compute_node_feature_importance(viz_model, benign_data)
    attack_imp = compute_node_feature_importance(viz_model, attack_data)

    # Optionally, sort nodes by total importance to keep the visualization compact
    def top_nodes_by_importance(imp: np.ndarray, max_nodes: int = 32):
        scores = imp.sum(axis=1)
        order = np.argsort(-scores)
        return imp[order[:max_nodes], :], order[:max_nodes]

    benign_imp_top, benign_nodes = top_nodes_by_importance(benign_imp, max_nodes=32)
    attack_imp_top, attack_nodes = top_nodes_by_importance(attack_imp, max_nodes=32)

    # Plot side-by-side heatmaps
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    for ax, imp, title, nodes in [
        (axes[0], benign_imp_top, "Benign window (gradient×input)", benign_nodes),
        (axes[1], attack_imp_top, "Attack window (gradient×input)", attack_nodes),
    ]:
        im = ax.imshow(imp, aspect="auto")
        ax.set_xlabel("Node feature")
        ax.set_ylabel("Node index (subset)")
        ax.set_title(title)
        ax.set_xticks(np.arange(len(feature_names)), labels=feature_names, rotation=45, ha="right")
        ax.set_yticks(np.arange(len(nodes)), labels=[str(int(n)) for n in nodes])
        fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)

    plt.tight_layout()
    attr_path = os.path.join(FIG_DIR, "fig_gnn_gradient_attributions.png")
    plt.savefig(attr_path, dpi=300, bbox_inches="tight")
    plt.close(fig)
    print(f"Saved gradient-based attribution heatmaps to: {attr_path}")


Loaded best config from checkpoint: {'batch_size': 32, 'epochs': 40, 'lr': 0.0002, 'lr_grid': [0.0002, 0.0003, 0.0005], 'weight_decay': 0.0001, 'hidden_dim': 32, 'num_layers': 2, 'dropout': 0.3, 'conv_type': 'gcn', 'aggr': 'mean', 'use_batchnorm': True, 'hidden_dim_grid': [32, 64], 'num_layers_grid': [1, 2, 3], 'dropout_grid': [0.0, 0.1, 0.3], 'aggr_grid': ['mean', 'mean+max'], 'conv_type_grid': ['sage', 'gcn'], 'mlp_hidden_dim': 64, 'mlp_layers': 2, 'num_classes': 1, 'early_stop': {'monitor': 'f1', 'mode': 'max', 'patience': 5, 'min_delta': 0.001}, 'warmup_steps': 200, 'total_steps': 10000}
Best validation metric stored: f1 = 0.22654218634316972

[VAL] Metrics at threshold 0.5 used during training:
{'acc': 0.3854185443503099, 'precision': 0.9908410929627551, 'recall': 0.1278913977223445, 'f1': 0.22654218634316972, 'loss': 4.576302989548515}

[VAL] Best F1 over PR curve ≈ 0.842 at threshold ≈ 0.0003

[VAL] Confusion matrix at threshold ≈ 0.0003:
TN: 2425 FP: 18940 FN: 37 TP: 50717

Sav